## Aim: To create a traing and test dataframes for downstream precesses

# Load & Extract Data

In [1]:
def load_data():
    # The following code contains the credentials for a file in your IBM Cloud Object Storage.
    # You might want to remove those credentials before you share your notebook.
    credentials_1 = {
        'IAM_SERVICE_ID': 'iam-ServiceId-18f05839-f64e-476f-a565-2a40dc0b9010',
        'IBM_API_KEY_ID': 'uq8s0oFeBFgGIdqEAxAngmj3ZWKpDB-dmfJRrw1Oly4z',
        'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
        'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
        'BUCKET': 'courseraibmadvanceddatasciencecap-donotdelete-pr-r1kusvnwe7do2x',
        'FILE': 'champs-scalar-coupling_new.zip'
    }

    from ibm_botocore.client import Config
    import ibm_boto3
    def download_file_cos(credentials,local_file_name,key):  
        cos = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=credentials['IBM_API_KEY_ID'],
        ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
        ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
        config=Config(signature_version='oauth'),
        endpoint_url=credentials['ENDPOINT'])
        try:
            res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
        except Exception as e:
            print(Exception, e)
        else:
            print('File Downloaded')

    download_file_cos(credentials_1, 'champs-scalar-coupling_new.zip', 'champs-scalar-coupling_new.zip')

    !ls
    !unzip champs-scalar-coupling_new.zip
    !chmod 600 *.csv

In [2]:
load_data()
!ls

File Downloaded
champs-scalar-coupling_new.zip
Archive:  champs-scalar-coupling_new.zip
  inflating: mulliken_charges.csv    
  inflating: structures.csv          
  inflating: structures.zip          
  inflating: test.csv                
  inflating: train.csv               
  inflating: sample_submission.csv   
  inflating: magnetic_shielding_tensors.csv  
  inflating: potential_energy.csv    
  inflating: scalar_coupling_contributions.csv  
  inflating: dipole_moments.csv      
champs-scalar-coupling_new.zip	scalar_coupling_contributions.csv
dipole_moments.csv		structures.csv
magnetic_shielding_tensors.csv	structures.zip
mulliken_charges.csv		test.csv
potential_energy.csv		train.csv
sample_submission.csv


# Basic Transfomations

## Training data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
train=pd.read_csv('train.csv')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [5]:
structures=pd.read_csv('structures.csv')
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [6]:
charges=pd.read_csv('mulliken_charges.csv')
charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [7]:
structures_0=structures.rename(columns={'atom_index':'atom_index_0','atom':'atom_0','x':'x_0','y':'y_0','z':'z_0'})
structures_0.head()

,molecule_name,atom_index_0,atom_0,x_0,y_0,z_0
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [8]:
train=pd.merge(train,structures_0,on=['molecule_name','atom_index_0'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277


In [9]:
structures_1=structures.rename(columns={'atom_index':'atom_index_1','atom':'atom_1','x':'x_1','y':'y_1','z':'z_1'})
structures_1.head()

,molecule_name,atom_index_1,atom_1,x_1,y_1,z_1
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [10]:
train=pd.merge(train,structures_1,on=['molecule_name','atom_index_1'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [11]:
train.shape

(4658147, 14)

In [12]:
charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [13]:
charges_0=charges.rename(columns={'atom_index':'atom_index_0','mulliken_charge':'mulliken_charge_0'})
charges_0.head()

,molecule_name,atom_index_0,mulliken_charge_0
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [14]:
train=pd.merge(train,charges_0,on=['molecule_name','atom_index_0'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922


In [15]:
charges_1=charges.rename(columns={'atom_index':'atom_index_1','mulliken_charge':'mulliken_charge_1'})
charges_1.head()

,molecule_name,atom_index_1,mulliken_charge_1
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [16]:
train=pd.merge(train,charges_1,on=['molecule_name','atom_index_1'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689


In [17]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1',
       'x_1', 'y_1', 'z_1', 'mulliken_charge_0', 'mulliken_charge_1'],
      dtype='object')

In [18]:
train=train[['id', 'molecule_name', 'atom_index_0', 'atom_index_1',
              'atom_0', 'x_0', 'y_0', 'z_0', 
              'atom_1', 'x_1', 'y_1', 'z_1',
              'mulliken_charge_0', 'mulliken_charge_1',
              'type','scalar_coupling_constant']]

In [19]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074


In [20]:
train.shape

(4658147, 16)

## Test data

In [21]:
test=pd.read_csv('test.csv')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [22]:
test=pd.merge(test,structures_0,on=['molecule_name','atom_index_0'],how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0


In [23]:
test=pd.merge(test,structures_1,on=['molecule_name','atom_index_1'],how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


In [24]:
test['mulliken_charge_0']=np.nan
test['mulliken_charge_1']=np.nan

In [25]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN


In [26]:
test.shape

(2505542, 15)

In [27]:
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'atom_0',
       'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1'],
      dtype='object')

In [39]:
test=test[['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0',
       'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1','type']]

In [40]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [41]:
test.shape

(2505542, 15)

In [42]:
train.shape

(4658147, 16)

# Save Train and Test Data to disk

In [43]:
train.to_csv('train_etl.csv',index=False)
test.to_csv('test_etl.csv',index=False)

In [44]:
!ls

champs-scalar-coupling_new.zip	scalar_coupling_contributions.csv
connectivity_model_nn_03.h5	structures.csv
dipole_moments.csv		structures.zip
magnetic_shielding_tensors.csv	test.csv
mulliken_charges.csv		test_etl.csv
pairs_dict.pkl			train.csv
potential_energy.csv		train_etl.csv
sample_submission.csv


# Upload train and test data to the cloud object store

In [45]:
# The code was removed by Watson Studio for sharing.

In [46]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [47]:
upload_file_cos(credentials_2,'train_etl.csv','train_etl.csv')

File Uploaded


In [48]:
upload_file_cos(credentials_2,'test_etl.csv','test_etl.csv')

File Uploaded
